In [2]:
from openInterface.api_calls import generate_response
import json
from models.character import Character
from models.chapter import Chapter
from models.scene import Scene
from models.story import Story

story = Story()

In [43]:
def export_book():
    # Create a list to store all the scene texts
    all_scene_texts = []
    all_scene_texts.append("This is the beginning of the book.")
    all_scene_texts.append("Summary:")
    all_scene_texts.append(story.summary)
    all_scene_texts.append("\n\n")

    all_scene_texts.append("Acts:")
    for act in story.acts:
        all_scene_texts.append(act)
        all_scene_texts.append("\n\n")

    all_scene_texts.append("Plotlines:")
    for plotline in story.plotlines:
        all_scene_texts.append(plotline)
        all_scene_texts.append("\n\n")

    all_scene_texts.append("Characters:")        
    for character in story.characters:
        all_scene_texts.append(character.name)
        all_scene_texts.append("\n")
        all_scene_texts.append(str(character.__dict__))

    for chapter in story.chapters:
        all_scene_texts.append(f"CHAPTER: {chapter.id}\n" + chapter.name)
        all_scene_texts.append(chapter.description)
        all_scene_texts.append("\n")
        
        # all_scene_texts.append("Characters:")
        # for character in chapter.characters:
        #     all_scene_texts.append(character.name)

        all_scene_texts.append("Goals:")
        all_scene_texts.append(chapter.goals)
        all_scene_texts.append("\n")

        all_scene_texts.append(f"SCENES: {len(chapter.scenes)}")
        for scene in chapter.scenes:
            all_scene_texts.append(f"Scene {scene.id}: " + scene.name)
            all_scene_texts.append("Description:")
            all_scene_texts.append(scene.description)
            # all_scene_texts.append("Characters:")
            # all_scene_texts.append(scene.characters).join(", ")
            all_scene_texts.append("Goals:")
            all_scene_texts.append(scene.goals)

            if hasattr(scene, 'final_text'):
                all_scene_texts.append("Scene:")
                all_scene_texts.append(scene.final_text)

        all_scene_texts.append("----------------------")
        all_scene_texts.append("\n")

    # Write the scene texts to a text file
    output_file = "book.txt"
    with open(output_file, "w") as f:
        for scene_text in all_scene_texts:
            f.write(scene_text + "\n")

    print(f"Scene texts written to {output_file}")


# 1- User writes a One-Paragraph Summary

In [4]:
story.summary = "In the blood-soaked realm of “Sword and Sinew,” where treachery festers like a poisoned chalice, Lady Seraphina, a rogue sorceress with a penchant for vengeance, embarks on a perilous quest. Her mission: to retrieve the Crown of Shadows, an artifact said to grant dominion over life and death. But standing in her way are rival warlords, a duplicitous court jester, and a band of undead knights who hunger for her soul. As the moon waxes and secrets unravel, Lady Seraphina dances on the edge of damnation, wielding her blade and dark magic in a deadly waltz toward destiny."

# 2- Character Summaries: Create brief character descriptions.

In [5]:
def create_character_objects(characterData):
    """
    Converts a dictionary of character data into an array of Character objects.

    Args:
        characterData (dict): A dictionary containing character data.

    Returns:
        list: List of Character objects.
    """
    characters = []
    for char_data in characterData['characters']:
        char_obj = Character()
        char_obj.name = char_data.get('Name', '')
        char_obj.role = char_data.get('Role', '')
        char_obj.age = char_data.get('Age', '')
        char_obj.occupation = char_data.get('Occupation', '')
        char_obj.appearance = char_data.get('Appearance', '')
        char_obj.personality_traits = char_data.get('Personality Traits', '')
        char_obj.motivation = char_data.get('Motivation', '')
        char_obj.backstory = char_data.get('Backstory', '')
        characters.append(char_obj)
    return characters

In [6]:
characters = []

## WE COULD HAVE CHARACTER ROLES HERE: ANTAGONISTS; PROTAGONISTS; ETC.
## THIS COULD RUN 2 TIMES; SO WE HAVE EXTRA CHARACTERS.
## TODO: fix the output of this prompt to make it error free when decoding
prompt = f"""
Imagine a story with this summary: '{story.summary}'. I need you to create brief character descriptions for this story. Feel free to add more characters or modify the prompts as needed. Please write a dict of characters in the following JSON compatible format, without any other text:
    {{
        "Name": "",
        "Role": "",
        "Age": "",
        "Occupation": "",
        "Appearance": "",
        "Personality Traits": "",
        "Motivation": "",
        "Backstory": ""
    }}
"""

response = generate_response(prompt)

charactersData = json.loads(response)
decodedCharacters = create_character_objects(charactersData)
characters += decodedCharacters

# # We save the character ids in the story object as well
for character in characters:
    story.characters.append(character)

# 3- Character Charts: Develop detailed character profiles.

In [49]:
def get_character_profile(character, summary):
    """
    Generates a detailed character profile for the given character.

    Args:
        character (object): An object representing the character.

    Returns:
        character'detailedProfile'.
    """

    prompt = f"""
        Consider the story '{summary}'. I need you to write a detailed character profile for this character '{character}' who will appear in it. 
    """

    responseData = generate_response(prompt)
    return responseData

def get_character_keyScenes(character, summary):
    """
    Generates a detailed character profile for the given character.

    Args:
        character (object): An object representing the character.

    Returns:
        character'detailedProfile'.
    """

    prompt = f"""
        Consider the story '{summary}'. I need you to write list a few key scenes For this character '{character}' where you think they can play a significant role in the story. Please separate each scene with the a new line and the word "SCENE: "
    """

    responseData = generate_response(prompt)
    return responseData

# Iterate through all characters and update their profiles
for character in characters:
    char = character.to_dict()
    character.detailedProfile = get_character_profile(char, story.summary)
    character.keyScenes = get_character_keyScenes(char, story.summary)

KeyboardInterrupt: 

# 4- Plot Summary: Outline the major plot points.

In [10]:
char_dicts = []
for character in characters:
    char_dicts.append(character.to_dict())

prompt = f"""
    Brainstorm and write the major plot points for a story that has this summary '{story.summary}', and involves these characters: {char_dicts}, considering the keyScenes that they have to participate. Use the Hero's Journey template, by Joseph Campbell. Make sure your answer is divided in 3 main acts, each one of them according to traditional creative writing rules. Your answer must be just 3 paragraphs, one for each act, separated by a blank line. Don't include titles for the acts, or an extra line with Act# in the beginning of each one.
"""

storyActs = generate_response(prompt).split('\n\n')

story.acts = storyActs

# 5- Create chapters out of the Acts


In [26]:
def generate_chapter(act):
    prompt = f"""
            Given the story act {act} that follows Joseph Campbell's Hero Journay, please write several story chapter descriptions for the specific act, including the major plot points and all the key scenes you think are important to make sure the story makes sense. Don't forget that the story is about "{story.summary}". Include the main characters, and add conflict between them and the story. Be innovative and creative. Make a unique story. Each chapter description should be a single paragraph separated by a single linebreak. Don't add separators.
        """

    story_chapters = generate_response(prompt).split('\n\n')

    print(story_chapters)

    for response in story_chapters:
        chapter = Chapter()
        chapter.description = response
        story.chapters.append(chapter)

story.chapters = []

for act in story.acts:
    generate_chapter(act)

['Chapter 1: Lady Seraphina, armed with dark magic and a deadly blade, crosses the Threshold into the blood-soaked realm of "Sword and Sinew." She faces her first challenge in the form of the rival warlord Lord Aric, who also seeks the Crown of Shadows for his nefarious purposes. Their clash sets the stage for a dangerous game of cat and mouse as Lady Seraphina navigates the treacherous landscape filled with betrayal and ambushes.', "Chapter 2: Lady Seraphina encounters the duplicitous court jester Malvolio, who schemes to manipulate her for his own gain. Torn between trusting him and seeing through his deception, Lady Seraphina must tread carefully as she uncovers more layers of treachery and deceit surrounding the Crown of Shadows. Malvolio's motives remain unclear as they dance around each other in a dangerous game of manipulation.", 'Chapter 3: The tension between Lady Seraphina and Lord Aric reaches its peak as they engage in a deadly battle for control of the Crown of Shadows. Wi

### Expanding chapter description to other elements

In [28]:
for chapter in story.chapters:
    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        Write a title for the chapter.
    """
    chapter.name = generate_response(prompt)

    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        Write the goal for the chapter in terms of narrative design and plot.
    """

    chapter.goals = generate_response(prompt)

    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        Write the consequences of the chapter that need to happen in the next chapter.
    """

    chapter.consequences = generate_response(prompt)

    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        Write the end result of the chapter in terms of story, including a plot twist.
    """

    chapter.end_result = generate_response(prompt)

    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        Write the name of the characters that participate in this chapter. Return just the names separated by a comma.
    """

    chapter.characters = generate_response(prompt).strip("[]").split(", ")

# 6- Scene List: Create a list of scenes in the novel and write them

In [45]:
def create_scenes_from_chapter(chapter):
    prompt = f"""
        You are outlining a novel. 
        This is the summary of the nove: {story.summary}. 
        You will brainstorm and write a logline for scenes for this specific chapter :{chapter.__dict__}. 
        I need you to write a list of scenes that will go from start to end of the chapter, describing what happens in the scene, who participates in it, what are their goals, what are the consequences for the next chapter and what is the end result of the scene. 
        
        Please write as many scenes as you can. Make them creative, detailed and inspiring. 
        
        Please return each scene in a single paragraph, without any headings, or separators, just a single linebreak.
    """

    return generate_response(prompt).split(sep='\n\n')

In [46]:
def define_scene_details(chapter, scene):
    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        This is the description of the scene you have to work with: {scene.__dict__}.
        Write a title for the scene.
    """
    
    scene.name = generate_response(prompt)

    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        This is the description of the scene you have to work with: {scene.__dict__}.
        Write the goals for this scene.
    """
    
    scene.goals = generate_response(prompt)

    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        This is the description of the scene you have to work with: {scene.__dict__}.
        Write the consequences of this scene for the story.
    """
    
    scene.consequences = generate_response(prompt)

    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        This is the description of the scene you have to work with: {scene.__dict__}.
        Write how this scene must end, including plot twists.
    """
    
    scene.end_result = generate_response(prompt)

    prompt = f"""
        This is a summary of a book: {story.summary}.
        This is the description of the chapter: {chapter.__dict__}.
        This is the description of the scene you have to work with: {scene.__dict__}.
        Write a narrative scene, in the style of a book, as if you were telling the story as a professional writer.
    """
    
    scene.final_text = generate_response(prompt)

In [47]:
previous_scene = ""
for chapter in story.chapters:
    print(f"Starting chapter {chapter.name}")
    sceneResponses = create_scenes_from_chapter(chapter)
    for sceneResponse in sceneResponses:
        print(f"Creating scene {sceneResponse}")
        scene = Scene()
        scene.description = sceneResponse
        scene.act = chapter.act
        scene.plotpoint = chapter.plotpoint
        scene.previous_scene = previous_scene
        scene.goals = chapter.goals
        scene.consequences = chapter.consequences
        scene.plotpoint = chapter.plotpoint
        scene.chapter = chapter.name
        # define_scene_details(scene, chapter)
        chapter.scenes.append(scene)

        previous_scene = "Before this scene, this happened: " + scene.description + scene.consequences

Starting chapter "Dancing on the Edge of Destiny"
Creating scene Scene 1: Lady Seraphina cautiously navigates the shadowed woods, her senses attuned to any hint of danger, as she comes face to face with a group of undead knights led by a vengeful wraith. Her goal is to defeat the undead and continue her journey, but the consequence is that the wraith reveals a cryptic message about the Crown of Shadows, leaving Lady Seraphina unsettled as she presses forward.
Creating scene Scene 2: Lady Seraphina confronts Lord Aric in a dramatic showdown, their blades clashing in a deadly dance of skill and power. Both seek the Crown of Shadows, but only one can emerge victorious. The consequence of this encounter is that Lady Seraphina gains valuable insight into Lord Aric's motives and learns that he will stop at nothing to claim the artifact for himself, setting the stage for a tense rivalry between them in the chapters to come.
Creating scene Scene 3: The duplicitous court jester, disguised in a 

# 9- Write the Novel: How can we make the final_text be longer, bigger and uncut?

In [ ]:
# chapters = []

# for scene in scenes:
#     prompt = f"""
#         Considering the summary of this scene {scene}, which belongs to this act of the story {act}, and the characters {characters}, write the scene.
#     """

#     chapters.append(api_calls(prompt))

# 10- Join all the chapters into a txt and export it.

In [48]:
export_book()

Scene texts written to book.txt


In [38]:
print(chapters[6].scenes[42].final_text)